In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from datetime import datetime
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import joblib 
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(r"C:\Users\sachi.mehta\Data Science\Customer segmentation\dataset.csv",encoding = 'unicode_escape')

In [3]:
df_fix = df[df['CustomerID'].notna()]
#df_fix = df.sample(10000, random_state = 42)
df_fix["InvoiceDate"] = pd.to_datetime(df_fix["InvoiceDate"],errors='coerce')
df_fix["TotalSum"] = df_fix["Quantity"] * df_fix["UnitPrice"]
snapshot_date = max(df_fix.InvoiceDate) + datetime.timedelta(days=1)
customers = df_fix.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalSum': 'sum'})
customers.rename(columns = {'InvoiceDate': 'Recency',
                            'InvoiceNo': 'Frequency',
                            'TotalSum': 'MonetaryValue'}, inplace=True)
pd.Series(np.cbrt(customers['MonetaryValue'])).values
customers_fix = pd.DataFrame()
customers_fix["Recency"] = stats.boxcox(customers['Recency'])[0]
customers_fix["Frequency"] = stats.boxcox(customers['Frequency'])[0]
customers_fix["MonetaryValue"] = pd.Series(np.cbrt(customers['MonetaryValue'])).values
scaler = StandardScaler()
scaler.fit(customers_fix)
customers_normalized = scaler.transform(customers_fix)
pd.DataFrame(customers_normalized)
sse = {}
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(customers_normalized)
    sse[k] = kmeans.inertia_
model = KMeans(n_clusters=3, random_state=42)
model.fit(customers_normalized)
customers["Cluster"] = model.labels_
customers.groupby('Cluster').agg({
    'Recency':'mean',
    'Frequency':'mean',
    'MonetaryValue':['mean', 'count']}).round(2)


Recency Frequency MonetaryValue      
           mean      mean          mean count
Cluster                                      
0         42.53     67.25       1005.31  1938
1        185.34     19.20        351.20  1651
2         17.89    312.64       7371.57   783

In [4]:
joblib.dump(model, 'customer_segmentation.pkl') 

['customer_segmentation.pkl']